In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import time, cv2
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras import backend as K
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16

In [2]:
#télécharger les datasets
X_train = pd.read_csv("/Users/ayseaylinkaya/Desktop/Datascientist/2 Project Rakuten/X_train_update.csv") 
X_test = pd.read_csv("/Users/ayseaylinkaya/Desktop/Datascientist/2 Project Rakuten/X_test_update.csv") 
y_train = pd.read_csv("/Users/ayseaylinkaya/Desktop/Datascientist/2 Project Rakuten/Y_train_CVw08PX.csv")

In [3]:
#fusionner les training sets pour analyser les articles par classe
X_train = X_train.sort_values(by = 'Unnamed: 0', ascending = True)
y_train = y_train.sort_values(by = 'Unnamed: 0', ascending = True)
part2 = X_train[X_train.columns[3:]]
df = pd.concat([y_train,part2], axis=1)
df["image_name"] = "image_" + df.imageid.map(str)+ "_product_" + df.productid.map(str) + ".jpg"
df["image_name"] = df["image_name"].astype(str)
df['fullpath']= '/Users/ayseaylinkaya/Desktop/Datascientist/2 Project Rakuten/images/image_train/'+ df['image_name']
df.head()

,Unnamed: 0,prdtypecode,productid,imageid,image_name,fullpath
0,0,10,3804725264,1263597046,image_1263597046_product_3804725264.jpg,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...
1,1,2280,436067568,1008141237,image_1008141237_product_436067568.jpg,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...
2,2,50,201115110,938777978,image_938777978_product_201115110.jpg,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...
3,3,1280,50418756,457047496,image_457047496_product_50418756.jpg,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...
4,4,2705,278535884,1077757786,image_1077757786_product_278535884.jpg,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...


In [4]:
df['class'] = df['prdtypecode']

df = df.replace({'class': {10: 1, 2280:2,   50:3, 1280:4, 2705:5, 2522:6, 2582:7, 1560:8, 1281:9, 1920:10, 2403:11,
       1140:12, 2583:13, 1180:14, 1300:15, 2462:16, 1160:17, 2060:18,   40:19,   60:20, 1320:21, 1302:22,
       2220:23, 2905:24, 2585:25, 1940:26, 1301:0}})

In [5]:
data = df.drop(["Unnamed: 0","prdtypecode", "imageid", "image_name"], axis=1)
data.head()

,productid,fullpath,class
0,3804725264,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...,1
1,436067568,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...,2
2,201115110,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...,3
3,50418756,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...,4
4,278535884,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...,5


In [6]:
#data_train, data_test = train_test_split(data, test_size = 0.2, random_state = 123)

In [8]:
train_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)
#test_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

batch_size = 32
data["class"] = data["class"].astype(str)
#data_test["class"] = data_test["class"].astype(str)

train_generator = train_data_generator.flow_from_dataframe(dataframe=data,
                                                           directory="",
                                                           x_col = "fullpath",
                                                           class_mode ="sparse",
                                                           target_size = (224 , 224),
                                                           zoom_range = [0.5,1.0],
                                                           batch_size = batch_size)

#test_generator = test_data_generator.flow_from_dataframe(dataframe=data_test,
#                                                         directory="",
#                                                         x_col = "fullpath",
#                                                         class_mode ="sparse",
#                                                         target_size = (224,224), 
#                                                         batch_size = batch_size,
#                                                         shuffle = False)

Found 84916 validated image filenames belonging to 27 classes.


In [9]:
base_model = VGG16(weights='imagenet', include_top=False)

for layer in base_model.layers: 
    layer.trainable = False

model = Sequential()

model.add(base_model) 

model.add(GlobalAveragePooling2D())
model.add(Dense(1024,activation='relu'))

model.add(Dropout(rate=0.2))
model.add(Dense(512, activation='relu'))

model.add(Dropout(rate=0.2))
model.add(Dense(27, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 27)                1

In [17]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

my_callbacks = [
    EarlyStopping(monitor = 'loss', patience = 3, mode = 'min', restore_best_weights = True),
    ModelCheckpoint(filepath='/Users/ayseaylinkaya/Desktop/Datascientist/2 Project Rakuten/model_image_all.{epoch:02d}-{loss:.2f}.h5'),
]

In [ ]:
history = model.fit_generator(generator=train_generator, 
                                epochs = 10,
                                steps_per_epoch = len(data)//batch_size,
                                callbacks=my_callbacks)

Epoch 1/10
2653/2653 [==============================] - 11192s 4s/step - loss: 1.4474 - acc: 0.5669
Epoch 2/10
2653/2653 [==============================] - 10560s 4s/step - loss: 1.3743 - acc: 0.5859
Epoch 3/10
2653/2653 [==============================] - 10447s 4s/step - loss: 1.3233 - acc: 0.5974
Epoch 4/10
2653/2653 [==============================] - 10457s 4s/step - loss: 1.2752 - acc: 0.6125
Epoch 5/10
2653/2653 [==============================] - 10433s 4s/step - loss: 1.2335 - acc: 0.6237
Epoch 6/10
2653/2653 [==============================] - 10975s 4s/step - loss: 1.2039 - acc: 0.6299
Epoch 7/10
2415/2653 [==========================>...] - ETA: 20:19 - loss: 1.1700 - acc: 0.6415